# OGGM glacier climate simulator

App to initialize a simple glacier and look how it develops under changing climate settings.

## Import packages

import plotting libaries

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.streams import Stream, param
import panel as pn

hv.extension('bokeh')
pn.extension()

import OGGM packages

In [ ]:
# Constants for initialization
from oggm import cfg
cfg.initialize()

# OGGM models
from oggm.core.massbalance import LinearMassBalance
from oggm.core.flowline import FluxBasedModel, RectangularBedFlowline, TrapezoidalBedFlowline, ParabolicBedFlowline

# There are several solvers in OGGM core. We use the default one for this experiment
from functools import partial
FlowlineModel = partial(FluxBasedModel, min_dt=0, cfl_number=0.01)

and some other useful packages

In [ ]:
import numpy as np

## Define variables for model

some constants:

In [ ]:
# number of steps from bottem to top of glacier
nx = 200

# model grid spacing in m
map_dx = 100

# width of glacier (3 gridpoints = 300 m)
widths = np.zeros(nx) + 3.

# distance along glacier (x-axis of glacier profil plot) in km
distance_along_glacier = np.linspace(0, nx, nx) * map_dx * 1e-3

model variables:

In [ ]:
init_flowline = []
bed_rock_curve = []
mb_model = []
mb_curve = []

define ranges for slider:

In [ ]:
# glacier top height
glacier_top_height_max = 4000
glacier_top_height_min = 3000
glacier_top_height_default = 3500
glacier_top_height_range = np.arange(glacier_top_height_min,
                                     glacier_top_height_max + 1,
                                     100)

# glacier bottom height
glacier_bottom_height_max = 2000
glacier_bottom_height_min = 1000
glacier_bottom_height_default = 1500
glacier_bottom_height_range = np.arange(glacier_bottom_height_min,
                                        glacier_bottom_height_max + 1,
                                        100)

# ELA height
ELA_height_min = 2000
ELA_height_max = 4500
ELA_height_default = 3000
ELA_height_range = np.arange(ELA_height_min,
                             ELA_height_max + 1,
                             100)

# mass balance gradient 
mb_gradient_max = 15
mb_gradient_min = 1
mb_gradient_default = 4
mb_gradient_range = np.arange(mb_gradient_min,
                              mb_gradient_max + 1,
                              1)

## Panel for initializing a simple glacier flowline

should contain geometry of the glacier bed, ...

In [ ]:
# function for initializing a simple glacier flowline
def initializing_glacier_bed(arg=None): 
    global init_flowline
    global bed_rock_curve
    
    # define how the profile of the glacier bed should look like
    if bed_rock_profile.value == 'linear':
        bed_h = np.linspace(glacier_top.value, glacier_bottom.value, nx)
    elif bed_rock_profile.value == 'decreasing':
        pass
    elif bed_rock_profile.value == 'increasing':
        pass
    elif bed_rock_profile.value == 'jump':
        pass

    # create bed_rock_curve for glacier profil plot
    bed_rock_curve = hv.Curve((distance_along_glacier, bed_h),
                          'distance along glacier (km)', 'altitude (m)',
                          label='glacier bed').opts(color='black')
    
    # set glacier surface height to bed height because at the beginning there is no glacier
    surface_h = bed_h

    # initialize flowline with choosen shape of glacier
    if bed_shape.value == 'rectangular':
        init_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                               widths=widths, map_dx=map_dx)
    elif bed_shape.value == 'trapezoidal':
        pass
    elif bed_shape.value == 'parabolic':
        pass

defining some widgets

In [ ]:
bed_shape = pn.widgets.Select(name='bed shape',
                              options=['rectangular', 'trapezoidal', 'parabolic'])

bed_rock_profile = pn.widgets.Select(name='bedrock profile',
                                     options=['linear', 'decreasing', 'increasing', 'jump'])

glacier_top = pn.widgets.DiscreteSlider(name='glacier top height',
                                        options=list(glacier_top_height_range),
                                        value=glacier_top_height_default)

glacier_bottom = pn.widgets.DiscreteSlider(name='glacier bottom height',
                                           options=list(glacier_bottom_height_range),
                                           value=glacier_bottom_height_default)

init_glacier_bed_button = pn.widgets.Button(name='initializing glacier bed',
                                            button_type='primary')
init_glacier_bed_button.param.watch(initializing_glacier_bed, 'clicks');

put panel together and show it

In [ ]:
bed_rock_panel = pn.Column(glacier_top, glacier_bottom, bed_shape, bed_rock_profile,
                          init_glacier_bed_button)

bed_rock_panel

In [ ]:
bed_rock_curve

### Panel for modifying mass balance profile and plot of mass balance profile

change ELA and gradient, as a climate parameter, later maybe change temperature and precipitation and convert this changes into a mass balance profile

In [ ]:
# function to define and change massbalance model
def change_mb_model(arg=None):
    global mb_model
    global mb_curve
    
    mb_model = LinearMassBalance(ELA_height.value, grad=mb_gradient.value)
    
    max_glacier_range = np.linspace(glacier_bottom_height_min,
                                    glacier_top_height_max, nx)
    annual_mb = mb_model.get_annual_mb(max_glacier_range) * cfg.SEC_IN_YEAR
    mb_curve = hv.Curve((annual_mb, max_glacier_range),
                        'annual mass balance (m/yr)',
                        'altitude (m)') * hv.HLine(ELA_height.value).opts(line_dash='dashed')

create some widgets to change mass balance

In [ ]:
ELA_height = pn.widgets.DiscreteSlider(name='equilibrium line height',
                                       options=list(ELA_height_range),
                                       value=ELA_height_default)

mb_gradient = pn.widgets.DiscreteSlider(name='mass balance gradient',
                                        options=list(mb_gradient_range),
                                        value=mb_gradient_default)

change_mb_model_button = pn.widgets.Button(name='change mass balance profil',
                                            button_type='primary')
change_mb_model_button.param.watch(change_mb_model, 'clicks');

put panel together and show it

In [ ]:
mb_panel = pn.Column(ELA_height, mb_gradient, change_mb_model_button)
mb_panel

### Create plot for glacier height (dynamic plot)

showing surface glacier height, maybe also from the previous step,...

### Put Panels and plots together